In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import pickle
import sklearn
from sklearn import preprocessing
import sys

sys.path.insert(1, "./code/")
from Eval import Eval

print(tf.__version__)

1.12.0


In [2]:
traindata = pd.read_pickle('data/traindata.pickle')
valdata   = pd.read_pickle('data/valdata.pickle')
testdata  = pd.read_pickle('data/testdata.pickle')

def numerize(item):
    if item == 'MWS':
        return 0
    elif item == 'HPL':
        return 1
    elif item == 'EAP':
        return 2
    raise ValueError('Author not recognized:', item)
   
encoder = sklearn.preprocessing.LabelEncoder()
encoder.fit(traindata.author)


Y_train = tf.keras.utils.to_categorical(encoder.transform(traindata.author))#np.array([numerize(x) for x in list(traindata.author)])
Y_val = tf.keras.utils.to_categorical(encoder.transform(valdata.author))#np.array([numerize(x) for x in list(valdata.author)])
Y_test = tf.keras.utils.to_categorical(encoder.transform(testdata.author))#np.array([numerize(x) for x in list(testdata.author)])


In [3]:
encoder.classes_

array(['EAP', 'HPL', 'MWS'], dtype=object)

In [4]:
# textual feature engineering
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=STOPWORDS, max_features=1500)

X_train = train_text_feats = vectorizer.fit_transform(traindata.text).todense()
X_val = val_text_feats = vectorizer.transform(valdata.text).todense() 
X_test = test_text_feats = vectorizer.transform(testdata.text).todense() 


In [5]:

# # Create a sigmoid layer:
# tf.keras.layers.Dense(64, activation='sigmoid')
# # Or:
# tf.keras.layers.Dense(64, activation=tf.sigmoid)

# # A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
# tf.keras.layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# # A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
# tf.keras.layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# # A linear layer with a kernel initialized to a random orthogonal matrix:
# tf.keras.layers.Dense(64, kernel_initializer='orthogonal')

# # A linear layer with a bias vector initialized to 2.0s:
# tf.keras.layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

In [6]:
# inspired by keras docs example: https://www.tensorflow.org/guide/keras#input_numpy_data

model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
tf.keras.layers.Dense(100, activation='relu'),
# Add another:
tf.keras.layers.Dense(100, activation='relu'),
# Add a softmax layer with 10 output units:
tf.keras.layers.Dense(3, activation='softmax')])


In [7]:
# Configure a model for categorical classification.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy]
              )

numpy.ndarray

In [9]:

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))

Train on 15664 samples, validate on 1958 samples
Epoch 1/10
15664/15664 [==============================] - 3s 180us/step - loss: 0.8209 - categorical_accuracy: 0.6131 - val_loss: 0.6369 - val_categorical_accuracy: 0.7298
Epoch 2/10
15664/15664 [==============================] - 2s 144us/step - loss: 0.5849 - categorical_accuracy: 0.7583 - val_loss: 0.6159 - val_categorical_accuracy: 0.7339
Epoch 3/10
15664/15664 [==============================] - 2s 144us/step - loss: 0.5206 - categorical_accuracy: 0.7888 - val_loss: 0.6217 - val_categorical_accuracy: 0.7477
Epoch 4/10
15664/15664 [==============================] - 2s 146us/step - loss: 0.4516 - categorical_accuracy: 0.8177 - val_loss: 0.7041 - val_categorical_accuracy: 0.7308
Epoch 5/10
15664/15664 [==============================] - 2s 141us/step - loss: 0.3730 - categorical_accuracy: 0.8502 - val_loss: 0.8460 - val_categorical_accuracy: 0.7201
Epoch 6/10
15664/15664 [==============================] - 2s 140us/step - loss: 0.2957 - ca

In [10]:
# returns loss, accuracy
result = model.predict(X_test, verbose=2 )

In [11]:
label_lookup = {}
for idx, label in enumerate(encoder.classes_):
    label_lookup[idx] = label
label_lookup

def best_label(row, lookup):
    mx = max(row)
    return lookup[list(row).index(mx)]

preds = [best_label(x, label_lookup) for x in result]


In [12]:
accuracy = Eval.get_accuracy(preds, list(testdata.author))
print("Accuracy: ", accuracy)

Accuracy:  0.6898313745528871


In [13]:
import time
timenow = time.asctime( time.localtime(time.time()) )
print("Finished at: ", timenow)

Finished at:  Sun Dec  9 00:42:41 2018
